In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shap

from pathlib import Path
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier

import wfm

shap.initjs()


%matplotlib inline
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'context'

In [ ]:
root_path = Path().resolve().parent
input_path = root_path / "data" / "input"
images_path = root_path / "images"
output_path = root_path / "output"

input_data = wfm.preprocessing.get_input_data(input_path)
input_data.head()

In [ ]:
input_data.info()

In [ ]:
# input_data = input_data.fillna("0")

# --- Split data 
split_random_state = 42
X = (
    input_data.drop(columns=["wildfire", "id", "n_daño", "geometry", "orientacio", "mant_viv"])
    .pipe(lambda df: pd.DataFrame(df))
)
# X = X.select_dtypes("number")
# from sklearn import preprocessing
# target_encoder = preprocessing.LabelEncoder()
# target_encoder.fit(input_data["n_daño"])
# y = target_encoder.transform(input_data["n_daño"])
y = input_data["n_daño"]
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=split_random_state
)
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"{y_train.size} train records and {y_test.size} test records.")

In [ ]:
# # --- Preprocessing
# categorical_columns = X.select_dtypes("object").columns.tolist()
# numerical_columns = X.select_dtypes("number").columns.tolist()
# preprocessor = make_column_transformer(
#     (OneHotEncoder(handle_unknown="ignore"), categorical_columns),
#     remainder="passthrough"
# )

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [ ]:
categorical_transformer.fit_transform(input_data[["fact_agua"]])

In [ ]:
from sklearn.compose import make_column_selector as selector

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, selector(dtype_exclude="category")),
        ('cat', categorical_transformer, selector(dtype_include="category"))
    ]
)
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])


In [ ]:
from sklearn import set_config

set_config(display='diagram')
model

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
_ = model.fit(X_train, y_train)
y_pred_test = model.predict(X_test)
print(
    classification_report(
        y_test,
        y_pred_test,
    )
)

In [ ]:
# # --- Model
# model = make_pipeline(
#     # imputer,
#     preprocessor,
#     RandomForestClassifier(
#         n_estimators=10,
#         n_jobs=-1,
#         random_state=42,
#     )
# )

# _ = model.fit(X_train, y_train)
# y_pred_test = model.predict(X_test)
# print(
#     classification_report(
#         y_test,
#         y_pred_test,
#     )
# )

In [ ]:
from sklearn.metrics import plot_confusion_matrix

In [ ]:
plot_confusion_matrix(model, X, y) 

In [ ]:
explainer = shap.Explainer(model.predict, X)

In [ ]:
X.iloc[i, :]

In [ ]:
# explainer = shap.TreeExplainer(model)
# explainer = 
for i in range(X.shape[0]):
    try:
        shap_values = explainer.shap_values(X.iloc[i:])
        print("i")
    except:
        pass

In [ ]:
explainer = shap.Explainer(model.predict, X)
shap_values = explainer.shap_values(X)

In [ ]:
explainer.feature_names

In [ ]:
shap.summary_plot(shap_values, X, show=False)

In [ ]:
model.named_steps

In [ ]:
model.steps

In [ ]:
type(model)

In [ ]:
type(model["preprocessor"])

In [ ]:
model["preprocessor"].named_transformers_["cat"]

## Simple Feature Importance

In [ ]:
model["classifier"].feature_importances_

In [ ]:
model.steps

In [ ]:
len(model["classifier"].feature_importances_)

In [ ]:
X.shape

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
categorical_columns = [
    'material',
 'orientacio',
 'prep_vivie',
 'mant_viv',
 'acceso_equ',
 'ac_supresi',
 'fact_agua'
]
numerical_columns = wfm.NUM_COLUMNS

In [ ]:
ohe = (model.named_steps['preprocessor']
         .named_transformers_['cat'])
feature_names = ohe.get_feature_names(input_features=categorical_columns)
feature_names = np.r_[feature_names, numerical_columns]

tree_feature_importances = (
    model.named_steps['classifier'].feature_importances_)
sorted_idx = tree_feature_importances.argsort()

y_ticks = np.arange(0, len(feature_names))
fig, ax = plt.subplots(figsize=(8, 8))
ax.barh(y_ticks, tree_feature_importances[sorted_idx])
ax.set_yticklabels(feature_names[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
plt.show()

In [ ]:
result = permutation_importance(model, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=3)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots(figsize=(8, 8))
ax.boxplot(result.importances[sorted_idx].T,
           vert=False, labels=X_test.columns[sorted_idx])
ax.set_title("Permutation Importances (test set)")
fig.tight_layout()
plt.show()

In [ ]:
result = permutation_importance(model, X_train, y_train, n_repeats=10,
                                random_state=42, n_jobs=3)
sorted_idx = result.importances_mean.argsort()

fig, ax = plt.subplots(figsize=(8, 8))
ax.boxplot(result.importances[sorted_idx].T,
           vert=False, labels=X_train.columns[sorted_idx])
ax.set_title("Permutation Importances (test set)")
fig.tight_layout()
plt.show()

In [ ]:
feature_importances = pd.DataFrame(
    model["classifier"].feature_importances_,
    columns=['Coefficients'],
    index=X.columns
)

feature_importances.sort_values("Coefficients").plot(kind='barh', figsize=(9, 7))
plt.title('Random Forest')
plt.axvline(x=0, color='.5')
# plt.subplots_adjust(left=.3)

In [ ]:
from sklearn.model_selection import cross_validate, RepeatedKFold, RepeatedStratifiedKFold


In [ ]:
cv_model = cross_validate(
    model,
    X,
    y,
    scoring=["r2", "neg_root_mean_squared_error"],
    cv=RepeatedStratifiedKFold(n_splits=4, n_repeats=30, random_state=42),
    return_train_score=True,
    return_estimator=True,
    n_jobs=-1
)
coefs = pd.DataFrame(
    [est.feature_importances_ for est in cv_model['estimator']],
    columns=X.columns
)

In [ ]:
model.ste

In [ ]:
# All scorer objects follow the convention that higher return values are better than lower return values. Thus metrics which measure the distance between the model and the data, like metrics.mean_squared_error, are available as neg_mean_squared_error which return the negated value of the metric.

In [ ]:
cv_scores = pd.DataFrame({"Train": cv_model['train_r2'], "Test": cv_model['test_r2']})
plt.figure(figsize=(10, 9))
sns.boxplot(data=cv_scores, orient='v', color='cyan', saturation=0.5)
plt.ylabel('Score')
plt.title('R2 Score for train and test sets')
# plt.subplots_adjust(left=.3)

In [ ]:
cv_scores = pd.DataFrame({"Train": cv_model['train_neg_root_mean_squared_error'], "Test": cv_model['test_neg_root_mean_squared_error']})
plt.figure(figsize=(10, 9))
sns.boxplot(data=cv_scores, orient='v', color='cyan', saturation=0.5)
plt.ylabel('Score')
plt.title('RMSE for train and test sets')
# plt.subplots_adjust(left=.3)

In [ ]:
plt.figure(figsize=(12, 9))
# sns.swarmplot(data=coefs, orient='h', color='k', alpha=0.5)
sns.boxplot(data=coefs, orient='h', color='cyan', saturation=0.5)
plt.axvline(x=0, color='.5')
plt.xlabel('Feature importance')
plt.title('Feature importance and its variability')
# plt.subplots_adjust(left=.3)

In [ ]:
shap_kernel_explainer = shap.KernelExplainer(model, X_train)
# shap_values_single = shap_kernel_explainer.shap_values(x_test.iloc[0,:])
# shap.force_plot(shap_kernel_explainer.expected_value[0],np.array(shap_values_single[0]), x_test.iloc[0,:],link='logit')